            # **Which crop would be the best to decide on with our field data?**  

Measuring essential soil metrics such as nitrogen, phosphorous, potassium levels, and pH value is an important aspect of assessing soil condition. However, it can be an expensive and time-consuming process, which can cause farmers to prioritize which metrics to measure based on their budget constraints.

Farmers have various options when it comes to deciding which crop to plant each season. Their primary objective is to maximize the yield of their crops, taking into account different factors. One crucial factor that affects crop growth is the condition of the soil in the field, which can be assessed by measuring basic elements such as nitrogen and potassium levels. Each crop has an ideal soil condition that ensures optimal growth and maximum yield.

A farmer reached out to you as a machine learning expert for assistance in selecting the best crop for his field. They've provided you with a dataset called `soil_measures.csv`, which contains:

- `"N"`: Nitrogen content ratio in the soil
- `"P"`: Phosphorous content ratio in the soil
- `"K"`: Potassium content ratio in the soil
- `"pH"` value of the soil
- `"crop"`: categorical values that contain various crops (target variable).

In [1]:
# The Packages
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [3]:
crops = pd.read_csv("datalab.csv")
print(crops.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   N       2200 non-null   int64  
 1   P       2200 non-null   int64  
 2   K       2200 non-null   int64  
 3   ph      2200 non-null   float64
 4   crop    2200 non-null   object 
dtypes: float64(1), int64(3), object(1)
memory usage: 86.1+ KB
None


I have no missing data. Now let's see if our crop column has some typos.

In [4]:
print(crops["crop"].value_counts())

crop
rice           100
maize          100
jute           100
cotton         100
coconut        100
papaya         100
orange         100
apple          100
muskmelon      100
watermelon     100
grapes         100
mango          100
banana         100
pomegranate    100
lentil         100
blackgram      100
mungbean       100
mothbeans      100
pigeonpeas     100
kidneybeans    100
chickpea       100
coffee         100
Name: count, dtype: int64


So personally I find this a bit weird. It takes some time for an apple tree to make apples for example, so you're not going to change a decision of a crop when you are working with fruit trees. But let's continue to pretend here. Perhaps our farmer wants to grow something new in a field he just bought. Let's hope our dataset fit with the region the farmer is in (coconuts, bananas and mango trees don't grow in the north hemisphere for example). this is very important in a real life context. to make a prediction model we need to acquire field data from the region where our farmer is based at.  

In [5]:
print(crops.head())

    N   P   K        ph  crop
0  90  42  43  6.502985  rice
1  85  58  41  7.038096  rice
2  60  55  44  7.840207  rice
3  74  35  40  6.980401  rice
4  78  42  42  7.628473  rice


To ensure a proper sampling (here it seems that our data is ordered by crop) I will mix the entire dataset.

In [6]:
crops = crops.sample(frac=1, random_state = 1984)

I'm creating dummies for my "crop" target variable.

In [7]:
crops_dummies = pd.get_dummies(crops["crop"], drop_first=True)

And now I'm going to make my models for each crop using a logistic regression model and get the ROC_AUC score (probability of acurate predictions).

In [8]:
X=crops[["N","P","K","ph"]].values
predictive_feature = {"crop":[], "ROC_AUC":[]}
logreg = LogisticRegression()
for crop in crops_dummies :
    y=crops_dummies[crop].values
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state=1984)
    logreg = LogisticRegression()
    logreg.fit(X_train,y_train)
    y_pred_prob = logreg.predict_proba(X_test)[:,1]
    ROC_AUC = metrics.roc_auc_score(y_test,y_pred_prob)
    predictive_feature["crop"].append(crop)
    predictive_feature["ROC_AUC"].append(ROC_AUC)


In [9]:
predictive_feature_df = pd.DataFrame(predictive_feature)
predictive_feature_df.sort_values("ROC_AUC", ascending=False, inplace=True)
predictive_feature_df

,crop,ROC_AUC
15,orange,1.000000
5,cotton,0.998361
0,banana,0.998289
6,grapes,0.981438
14,muskmelon,0.980190
18,pomegranate,0.977447
20,watermelon,0.973961
8,kidneybeans,0.961690
17,pigeonpeas,0.957252
3,coconut,0.946322


Knowing the phosphore, potassium, nitrogen and pH level, I should be able, using these models, to tell our farmer whether he can grow or not (from orange to maize). Jute, rice, papaya cannot be predicted with enough confidence of success with these variables in hand (N, P, K, pH).    